In [1]:
import pandas as pd
import numpy as np
import json
import altair as alt
import os
#os.chdir("..")


Reading in the multisheet dataset

In [3]:
dfs = pd.read_excel("data/raw/ons_subnational.xlsx", None, skiprows=0)

In [4]:
cleanded_dfs = []
for df in dfs.values():
    # itterate down the rows until a row with 'Area Code' in the first column is found
    # this is the start of the data
    for i in range(len(df)):
        if df.iloc[i,0] == 'Area Code':
            df.columns = df.iloc[i]
            df = df.iloc[i+1:]
            cleanded_dfs.append(df)
            break

Concatting, tidying and melting:

In [9]:
df = pd.concat(cleanded_dfs)

id_vars = ['Area Code', 'Country', 'Nation', 'Region',
       'County or Unitary Authority', 'Local Authority District', 'Lower 95% Confidence Interval',
       'Upper 95% Confidence Interval', 'Notes',
       'ITL Level 2', 'ITL Level 3', 
       'Data accuracy']


df = df.melt(id_vars=id_vars, var_name='Series', value_name='Value')

# if Local Authority District is undefined, use County or Unitary Authority
df['Local Authority District'] = df['Local Authority District'].fillna(df['County or Unitary Authority'])

df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
df = df.dropna(subset=['Local Authority District', 'Value'])

df['median'] = df.groupby(['Series'])['Value'].transform('median')
df['std'] = df.groupby(['Series'])['Value'].transform('std')
df['z_score'] = (df['Value'] - df['median']) / df['std']

df['norm_value'] = df.Value/df['median']

series_to_keep = {
    "Gross Value Added (GVA) per hour worked (£)" : {
              "Theme": "Productivity",
              "name": "GVA per hour worked (£)",
              "format": "$,.2f",
              "scheme" : "interpolateWarm"
       },
    'Gross median weekly pay (£)' : {
              "Theme": "Income",
              "name": "Gross median weekly pay (£)",
              "format": "$,.2f"
       },
    'Employment rate, ages 16-64 (%)' : {
              "Theme": "Employment",
              "name": "Employment rate, ages 16-64 (%)",
              "format": ".2%"
       },
    'Average travel time in minutes to reach nearest large employment centre (500 to 4999 jobs available), by public transport or walking (minutes)' : {
              "Theme": "Transport",
              "name": "Average Public Transport Travel Time to Nearest Large Employment Centre (minutes)",
              "format": ".2f"
       },
    'Percentage of young people achieving GCSEs (and equivalent qualifications) in English and maths by age 19 (%)' : {
              "Theme": "Education",
              "name": "Percentage of Young People Achieving GCSEs in English and Maths by Age 19 (%)",
              "format": ".2%"
       },
    'Female Healthy Life Expectancy (years)' : {
              "Theme": "Health",
              "name" : "Female Healthy Life Expectancy (years)",
              "format" : ".2f"
       },
    'Male Healthy Life Expectancy (years)' : {
        "Theme": "Health",
        "name" : "Male Healthy Life Expectancy (years)",
       "format" : ".2f"
    },
    'Age-standardised mortality rate for those aged under 75 (per 100,000 population)': {
              "Theme": "Health",
              "name" : "Under 75 Age-standardised mortality rates",
              "format" : ".2f"
       },
    'Mean satisfaction with your life nowadays scored 0 (not at all) - 10 (completely)': {
              "Theme": "Wellbeing",
              "name" : "Mean Reported Life Satisfaction (0-10)",
              "format" : ".2f"
       },
    'Net additions per 1,000 stock': {
              "Theme": "Housing",
              "name" : "Net additions per 1,000 stock",
              "format" : ".2f"
       }
}

df = df[df.Series.isin(series_to_keep.keys())]
df['Theme'] = df.Series.apply(lambda x: series_to_keep[x]['Theme'])
df['format'] = df.Series.apply(lambda x: series_to_keep[x]['format'])
df['scheme'] = df.Series.apply(lambda x: series_to_keep[x]['scheme'] if 'scheme' in series_to_keep[x] else 'interpolateCool')
df.Series = df.Series.apply(lambda x: series_to_keep[x]['name'])
df['series_id'] = df['Series'].astype('category').cat.codes

df['rank'] = df.groupby(['Series'])['Value'].rank(ascending=False)
df['out_of'] = df.groupby(['Series'])['Value'].transform('count')

df[["Area Code", "Local Authority District", "Series", "Theme", "Value", "median", "z_score", "norm_value", "rank", "out_of", "series_id", "format", "scheme"]].to_json("/Users/finn/Documents/GitHub/eco4Map/data/cleaned/indicators.json", orient="records") 

In [18]:
df = pd.concat(cleanded_dfs)
df = df.dropna(subset=['Local Authority District'])
df[df['Local Authority District'].str.contains('Bristol')]

,Area Code,ITL Level 1,County or Unitary Authority,Local Authority District,Gross Value Added (GVA) per hour worked (£),Notes,Country,Nation,Region,Gross median weekly pay (£),...,"Age-standardised mortality rate for those aged under 75 (per 100,000 population)",Mean satisfaction with your life nowadays scored 0 (not at all) - 10 (completely),Data accuracy,Mean feeling things done in life are worthwhile scored 0 (not at all) - 10 (completely),Mean happiness yesterday scored 0 (not at all) - 10 (completely),Mean anxiety yesterday scored 0 (not at all) - 10 (completely),"Net additions per 1,000 stock",Police Force Area,Homicide Offences (per million population),% living in an area that has a devolution deal with a directly elected mayor


In [9]:
# how many rows for each Series
df.groupby(['Series'])['Value'].count()

Series
Average Public Transport Travel Time to Nearest Large Employment Centre (minutes)    270
Employment rate, ages 16-64 (%)                                                      303
Female Healthy Life Expectancy (years)                                               133
GVA per hour worked (£)                                                              315
Gross median weekly pay (£)                                                          314
Male Healthy Life Expectancy (years)                                                 133
Mean Reported Life Satisfaction (0-10)                                               311
Net additions per 1,000 stock                                                        250
Percentage of Young People Achieving GCSEs in English and Maths by Age 19 (%)        249
Under 75 Age-standardised mortality rates                                            249
Name: Value, dtype: int64

In [8]:
df.series_id.unique()

array([3, 4, 1, 0, 8, 2, 5, 9, 6, 7], dtype=int8)

In [23]:
df.Series.unique()

array(['Gross Value Added (GVA) per hour worked (£)',
       'Gross median weekly pay (£)', 'Employment rate, ages 16-64 (%)',
       'Gross disposable household income, per head (£)',
       'Average travel time in minutes to reach nearest large employment centre (500 to 4999 jobs available), by public transport or walking (minutes)',
       'Average travel time in minutes to reach nearest large employment centre (500 to 4999 jobs available), by car (minutes)',
       'Average travel time in minutes to reach nearest large employment centre (500 to 4999 jobs available), by cycle (minutes)',
       'Percentage of premises with gigabit-capable broadband (%)',
       'Percentage of 4G coverage by at least one mobile network operator (%)',
       'Percentage of pupils in state-funded schools meeting the expected standard in reading, writing and maths at the end of key stage 2 (%)',
       'Percentage of young people achieving GCSEs (and equivalent qualifications) in English and maths by ag

In [21]:
df = cleanded_dfs[5]
df

,Area Code,Country,ITL Level 1,ITL Level 2,Combined Authority or City Region,Total FDI international investment position abroad at end period (£ million),Notes
1,K02000001,United Kingdom,NaN,NaN,NaN,1769316,NaN
2,K99000001,NaN,UK not allocated,NaN,NaN,2597,NaN
3,TLC,NaN,North East,NaN,NaN,26878,NaN
4,TLC1,NaN,NaN,Tees Valley and Durham,NaN,16413,NaN
5,E47000006,NaN,NaN,NaN,Tees Valley,3249,NaN
...,...,...,...,...,...,...,...
64,na,NaN,NaN,NaN,Edinburgh and South East Scotland City Region,24227,NaN
65,TLM8,NaN,NaN,West Central Scotland,NaN,27271,NaN
66,na,NaN,NaN,NaN,Glasgow City Region,27470,NaN
67,TLM9,NaN,NaN,Southern Scotland,NaN,5742,NaN
